### Capstone Project Notebook

In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [22]:
CLIENT_ID = 'YTTMJU4QPACEPYYJ40DZGA3CZOG5KHSMKR2WROTSKEQFU0IM' # Foursquare ID
CLIENT_SECRET = 'NOK10UKBEBCHWUU1N22TF3ACZSNGIWWUMBZTIZBIW04NAFVM' # Foursquare Secret
VERSION = '20180605' # API version
LIMIT =100

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
urltext = requests.get(url).text
data = BeautifulSoup(urltext, 'html.parser')

# creat a new Dataframe
columnlabel = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = columnlabel)

# loop through to find postcode, borough, neighborhood 
content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# drop burroughs with none assigned
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

# group neighborhoods with like burroughs
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [11]:
df2['Borough'] = df['Borough'].str[:-1]
df2['Postalcode'] = df['Postalcode'].str[:-1]

In [12]:
df2.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn


In [13]:
df2.shape

(180, 3)

In [14]:
lats = pd.read_csv("C:/Users/kxbh820/Downloads/Geospatial_Coordinates.csv")

In [15]:
lats.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df3 = pd.merge(df2,lats,left_on = ["Postalcode"],right_on = ["Postal Code"],how='left')
df3.dropna(inplace = True)

In [17]:
df3.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,,NaN,NaN,NaN
1,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
3,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
4,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917


In [20]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
venues = getNearbyVenues(names=df3['Borough'],latitudes=df3['Latitude'],longitudes=df3['Longitude'])


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [28]:
venues.shape

(2133, 7)

In [29]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [31]:
venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,113,113,113,113,113,113
Downtown Toronto,1223,1223,1223,1223,1223,1223
East Toronto,124,124,124,124,124,124
East York,75,75,75,75,75,75
Etobicoke,76,76,76,76,76,76
Mississauga,12,12,12,12,12,12
North York,238,238,238,238,238,238
Scarborough,97,97,97,97,97,97
West Toronto,158,158,158,158,158,158


In [32]:

# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017699,0.000000,...,0.008850,0.000000,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850
1,Downtown Toronto,0.000000,0.000818,0.000818,0.000818,0.001635,0.001635,0.000818,0.013900,0.001635,...,0.009812,0.001635,0.000818,0.003271,0.000000,0.004906,0.000818,0.000000,0.000818,0.005724
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,0.000000,0.000000,0.016129
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004202,0.004202,0.000000,0.000000,0.000000,0.000000,0.000000,0.008403,0.000000,...,0.000000,0.004202,0.004202,0.008403,0.000000,0.000000,0.000000,0.000000,0.016807,0.000000
7,Scarborough,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,...,0.000000,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006329,...,0.018987,0.000000,0.000000,0.012658,0.000000,0.006329,0.000000,0.000000,0.000000,0.012658
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000


In [36]:
grouped.shape


(10, 269)

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Cluster Labels,Coffee Shop,Sandwich Place,Park,Café,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Clothing Store
1,Downtown Toronto,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Seafood Restaurant
2,East Toronto,Cluster Labels,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Park,Ice Cream Shop,Fast Food Restaurant,Restaurant
3,East York,Coffee Shop,Bank,Burger Joint,Pharmacy,Pizza Place,Park,Sporting Goods Shop,Sandwich Place,Beer Store,Indian Restaurant
4,Etobicoke,Cluster Labels,Pizza Place,Sandwich Place,Coffee Shop,Gym,Discount Store,Pharmacy,Grocery Store,Liquor Store,Fast Food Restaurant
5,Mississauga,Cluster Labels,Hotel,Coffee Shop,Burrito Place,Fried Chicken Joint,Intersection,Mediterranean Restaurant,Middle Eastern Restaurant,Sandwich Place,Gym
6,North York,Cluster Labels,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Park,Pizza Place,Sandwich Place,Fast Food Restaurant
7,Scarborough,Chinese Restaurant,Coffee Shop,Breakfast Spot,Fast Food Restaurant,Bakery,Bank,Indian Restaurant,Intersection,Pharmacy,Pizza Place
8,West Toronto,Cluster Labels,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Park,Pizza Place,Grocery Store
9,York,Cluster Labels,Park,Trail,Skating Rink,Sandwich Place,Field,Restaurant,Bus Line,Bar,Pool


In [37]:
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 0, 4, 1, 3, 0, 3, 2])

In [42]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017699,0.000000,...,Cluster Labels,Coffee Shop,Sandwich Place,Park,Café,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Clothing Store
1,Downtown Toronto,0.0,0.000818,0.000818,0.000818,0.001635,0.001635,0.000818,0.013900,0.001635,...,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Seafood Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,0.000000,...,Cluster Labels,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Park,Ice Cream Shop,Fast Food Restaurant,Restaurant
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Burger Joint,Pharmacy,Pizza Place,Park,Sporting Goods Shop,Sandwich Place,Beer Store,Indian Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,...,Cluster Labels,Pizza Place,Sandwich Place,Coffee Shop,Gym,Discount Store,Pharmacy,Grocery Store,Liquor Store,Fast Food Restaurant
